In [2]:
from torch import optim
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PySimpleGUI.PySimpleGUI import Window
import sys
import os
import PySimpleGUI as sg
import tkinter as tk
from Data_collection_train import TrainingDataFrameApp
from torch.utils.tensorboard import SummaryWriter


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Create the tkinter window
# window = tk.Tk()
# window.geometry("400x400")
# window.configure(bg="#f0f0f0")

# # Create an instance of the TrainingDataFrameApp class
# app = TrainingDataFrameApp(window)

# # Start the tkinter event loop
# window.mainloop()


class CustomDataset(Dataset):
    def __init__(self, df, num_columns):
        self.df = df
        self.num_columns = num_columns

    def _custom_logic(self, subset):
        cpu_variance = subset['cpu_uses'].var()
        memory_variance = subset['memory_uses'].var()
        disc_variance = subset['disc_uses'].var()
        wifi_variance = subset['Wi_fi_uses'].var()

        y_value = 0 
        if 1 in subset['Type'].values:
            y_value = 0
        else:
            # hibernate 
            if cpu_variance > 0.7 and memory_variance > 0.7 and disc_variance > 0.7 and wifi_variance > 0.7:
                y_value = .1
            # sleep
            elif cpu_variance > 0.5 and memory_variance > 0.5 and disc_variance > 0.5 and wifi_variance > 0.5 and \
                    cpu_variance < 0.7 and memory_variance < 0.7 and disc_variance < 0.7 and wifi_variance < 0.7:
                y_value = .2
            # shutoff display
            elif cpu_variance < 0.3 and memory_variance < 0.3 and disc_variance > 0.5 and wifi_variance < 0.5:
                y_value = .3
            # shutoff display
            elif cpu_variance < 0.3 and memory_variance < 0.3 and disc_variance < 0.5 and wifi_variance > 0.5:
                y_value = .3
        return y_value

    def __len__(self):
        return len(self.df) // self.num_columns

    def __getitem__(self, idx):
        start_row = idx * self.num_columns
        end_row = (idx + 1) * self.num_columns
        subset = self.df.iloc[start_row:end_row]
        y = self._custom_logic(subset)
        x = subset.values
        x = torch.tensor(x)
        x= x.reshape(1,250)

        if y is not None:
            y = torch.tensor(y)
        else:
            y = torch.tensor(-1)  # Default value when logic doesn't assign a specific value
        return x, y.unsqueeze(-1)



layout = [
    [sg.Text("Enter the name of the file you want to train:", font=("Arial", 12))],
    [sg.Input(key='-FILENAME-', size=(40, 1))],
    [sg.Text("Enter the number of epochs:", font=("Arial", 12))],
    [sg.Input(key='-NUM_EPOCHS-', size=(40, 1))],
    [sg.Button("Run", size=(10, 1), button_color=('white', '#4CAF50'), font=("Arial", 12))],
    [sg.Button("Exit", size=(10, 1), button_color=('white', '#D32F2F'), font=("Arial", 12))],
    [sg.Output(size=(80, 20), font=("Arial", 12), key='-OUTPUT-')]
]

# Create the PySimpleGUI window
window = sg.Window("Program Runner", layout)

if __name__ == '__main__':
    while True:
        event, values = window.read()
        if event == sg.WINDOW_CLOSED or event == 'Exit':
            break
        elif event == 'Run':
            file_name = values['-FILENAME-']
            file_path = file_name + '.csv'
            df = pd.read_csv(file_path)
            df = df.astype('float32')
            Dataset = CustomDataset(df, 50)
            dataloader = DataLoader(Dataset, batch_size=32,shuffle=False)

            model = nn.Sequential(
                nn.Flatten(), 
                nn.Linear(250,50),  # here 5 is the number of dataframe columns or input features
                nn.ReLU(),
                nn.Linear(50, 1),  
                nn.Sigmoid()
                
            ).to(device)

            optimizer = optim.Adam(model.parameters(), lr=0.001)
            loss_fn = nn.MSELoss()

            total_loss = 0
            num_batches = len(dataloader)
            NUM_EPOCHS = int(values['-NUM_EPOCHS-'])  # Retrieve NUM_EPOCHS from the input field
            print_interval = 1
            loop_counter = 0
            losses = [] 
            for epoch in range(NUM_EPOCHS):
                for batch_idx, (x, y) in enumerate(dataloader):
                    x = x.to(device)
                    y = y.float()
                    y = y.to(device)
                    model.train()
                    # Forward pass
                    y_pred = model(x)
                    # print(f'shape of x: {x.shape}')
                    # print(f'shape of y: {y.shape}')
                    # print(f'shape of y_pred: {y_pred.shape}')

                    # Compute the loss
                    loss = loss_fn(y_pred, y)
                    total_loss += loss.item()

                    # Backward pass and optimization
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    # Print training progress
                    if (batch_idx + 1) % print_interval == 0:
                        avg_loss = total_loss / print_interval
                        print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Batch [{batch_idx+1}/{num_batches}], Loss: {avg_loss:.4f}')
                        total_loss = 0
                        losses.append(loss.item())

                        # Save the model
                        save_path = f'your_model{loop_counter}.pt'
                        torch.save(model.state_dict(), save_path)
                    
                    loop_counter += 1
            variance = torch.var(torch.tensor(losses))

            std_dev = torch.sqrt(variance)
            print(f'Variance: {variance}')
            print(f'Standard deviation: {std_dev}')
            print("Training completed!")

        window['Run'].update(disabled=False)
        window.refresh()


window.close()



In [1]:
from torch import optim
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PySimpleGUI.PySimpleGUI import Window
import sys
import os
import PySimpleGUI as sg
import tkinter as tk
from Data_collection_train import TrainingDataFrameApp
from torch.utils.tensorboard import SummaryWriter


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Create the tkinter window
# window = tk.Tk()
# window.geometry("400x400")
# window.configure(bg="#f0f0f0")

# # Create an instance of the TrainingDataFrameApp class
# app = TrainingDataFrameApp(window)

# # Start the tkinter event loop
# window.mainloop()


class CustomDataset(Dataset):
    def __init__(self, df, num_columns):
        self.df = df
        self.num_columns = num_columns

    def _custom_logic(self, subset):
        cpu_variance = subset['cpu_uses'].var()
        memory_variance = subset['memory_uses'].var()
        disc_variance = subset['disc_uses'].var()
        wifi_variance = subset['Wi_fi_uses'].var()

        y_value = 0 
        if 1 in subset['Type'].values:
            y_value = 0
        else:
            # hibernate 
            if cpu_variance > 0.7 and memory_variance > 0.7 and disc_variance > 0.7 and wifi_variance > 0.7:
                y_value = .1
            # sleep
            elif cpu_variance > 0.5 and memory_variance > 0.5 and disc_variance > 0.5 and wifi_variance > 0.5 and \
                    cpu_variance < 0.7 and memory_variance < 0.7 and disc_variance < 0.7 and wifi_variance < 0.7:
                y_value = .2
            # shutoff display
            elif cpu_variance < 0.3 and memory_variance < 0.3 and disc_variance > 0.5 and wifi_variance < 0.5:
                y_value = .3
            # shutoff display
            elif cpu_variance < 0.3 and memory_variance < 0.3 and disc_variance < 0.5 and wifi_variance > 0.5:
                y_value = .3
        return y_value

    def __len__(self):
        return len(self.df) // self.num_columns

    def __getitem__(self, idx):
        start_row = idx * self.num_columns
        end_row = (idx + 1) * self.num_columns
        subset = self.df.iloc[start_row:end_row]
        y = self._custom_logic(subset)
        x = subset.values
        x = torch.tensor(x)
        x= x.reshape(1,250)

        if y is not None:
            y = torch.tensor(y)
        else:
            y = torch.tensor(-1)  # Default value when logic doesn't assign a specific value
        return x, y.unsqueeze(-1)



layout = [
    [sg.Text("Enter the name of the file you want to train:", font=("Arial", 12))],
    [sg.Input(key='-FILENAME-', size=(40, 1))],
    [sg.Text("Enter the number of epochs:", font=("Arial", 12))],
    [sg.Input(key='-NUM_EPOCHS-', size=(40, 1))],
    [sg.Text("Please Enter the Name of your pre trained model if you have one :", font=("Arial", 12))],
    [sg.Input(key='-PRE_TRAINED-', size=(40, 1))],
    [sg.Button("Run", size=(10, 1), button_color=('white', '#4CAF50'), font=("Arial", 12))],
    [sg.Button("Exit", size=(10, 1), button_color=('white', '#D32F2F'), font=("Arial", 12))],
    [sg.Output(size=(80, 20), font=("Arial", 12), key='-OUTPUT-')]
]

# Create the PySimpleGUI window
window = sg.Window("Program Runner", layout)

if __name__ == '__main__':
    while True:
        event, values = window.read()
        if event == sg.WINDOW_CLOSED or event == 'Exit':
            break
        elif event == 'Run':
            file_name = values['-FILENAME-']
            file_path = file_name + '.csv'
            df = pd.read_csv(file_path)
            df = df.astype('float32')
            Dataset = CustomDataset(df, 50)
            dataloader = DataLoader(Dataset, batch_size=32,shuffle=False)

            model = nn.Sequential(
                nn.Flatten(), 
                nn.Linear(250,50),  # here 5 is the number of dataframe columns or input features
                nn.ReLU(),
                nn.Linear(50, 1),  
                nn.Sigmoid()
                
            ).to(device)
            model_state_name = values['-PRE_TRAINED-']
            model_state_name = model_state_name + '.pt'

            if model_state_name != '':
                model.load_state_dict(torch.load(model_state_name))
            else:
                break

            optimizer = optim.Adam(model.parameters(), lr=0.001)
            loss_fn = nn.MSELoss()

            total_loss = 0
            num_batches = len(dataloader)
            NUM_EPOCHS = int(values['-NUM_EPOCHS-'])  # Retrieve NUM_EPOCHS from the input field
            print_interval = 1
            loop_counter = 0
            losses = [] 
            writer = SummaryWriter()
            for epoch in range(NUM_EPOCHS):
                for batch_idx, (x, y) in enumerate(dataloader):
                    x = x.to(device)
                    y = y.float()
                    y = y.to(device)
                    model.train()
                    # Forward pass
                    y_pred = model(x)

                    # Compute the loss
                    loss = loss_fn(y_pred, y)
                    total_loss += loss.item()

                    # Backward pass and optimization
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    # Print training progress
                    if (batch_idx + 1) % print_interval == 0:
                        avg_loss = total_loss / print_interval
                        total_loss = 0
                        variance = torch.var(torch.tensor(losses))
                        std_dev = torch.sqrt(variance)
                        losses.append(loss.item())
                        writer.add_scalar('Loss', avg_loss, epoch * num_batches + batch_idx)
                        writer.add_scalar('Variance', variance, epoch * num_batches + batch_idx)
                        # Save the model
                        save_path = f'your_model{loop_counter}.pt'
                        torch.save(model.state_dict(), save_path)
                    
                    loop_counter += 1
            writer.close()
            print("Thank you for your patience Training has been compleated !")
        window['Run'].update(disabled=False)
        window.refresh()


window.close()



In [2]:
import pandas as pd 
df = pd.read_csv('12.csv')
df

,cpu_uses,memory_uses,Wi_fi_uses,disc_uses,Type
0,0.000000,0.000000e+00,0.0,0.0,0.0
1,0.000000,1.053162e-02,0.0,0.0,0.0
2,0.000000,1.226657e-01,0.0,0.0,0.0
3,0.000000,0.000000e+00,0.0,0.0,1.0
4,0.000000,1.865272e-01,0.0,0.0,0.0
5,0.000000,2.971305e-03,0.0,0.0,0.0
6,0.000000,9.732949e-02,0.0,0.0,0.0
7,0.000000,9.023988e-05,0.0,0.0,0.0
8,0.004543,9.999999e-01,0.0,0.0,0.0
9,0.000000,1.045526e-03,0.0,0.0,0.0


In [3]:
len(df)

18

In [65]:
model.eval()
with torch.no_grad():
    for batch_idx, (x, y) in enumerate(dataloader):
        x = x.to(device)
        y = y.float() 
        y = y.to(device)
        y_pred = model(x)
        print(y_pred)
        break

tensor([[[1.2714e-04],
         [6.1944e-05],
         [2.1338e-04],
         [7.0907e-05],
         [1.7519e-04]],

        [[7.0177e-05],
         [7.2420e-05],
         [7.8432e-05],
         [7.2725e-05],
         [8.4925e-05]],

        [[7.1332e-05],
         [7.2189e-05],
         [6.4459e-05],
         [1.4727e-04],
         [1.4595e-04]],

        [[8.0511e-05],
         [6.8218e-05],
         [7.1563e-05],
         [7.2725e-05],
         [1.7509e-04]],

        [[1.4545e-04],
         [3.1680e-05],
         [7.2189e-05],
         [7.8436e-05],
         [6.1926e-05]],

        [[7.2420e-05],
         [1.4171e-04],
         [7.0143e-05],
         [1.6354e-04],
         [7.0907e-05]],

        [[1.2714e-04],
         [8.4925e-05],
         [7.0125e-05],
         [7.1842e-05],
         [6.8203e-05]],

        [[6.1851e-05],
         [1.7536e-04],
         [7.8425e-05],
         [1.4636e-04],
         [7.2420e-05]],

        [[6.1926e-05],
         [7.2189e-05],
         [6.2052e-

In [46]:
x,y = CustomDataset.__getitem__(0)

In [41]:
model = nn.Sequential(
                nn.Linear(5, 30),
                nn.ReLU(),
                nn.Linear(30, 256),
                nn.ReLU(),
                nn.Linear(256, 1),
                nn.Tanh()
).to(device)

tensor([0], device='cuda:0')


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>

In [30]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the loss function
loss_fn = nn.MSELoss()

# Training loop
for x, y in dataloader:
    x = x.to(device)
    y = y.float()  # Convert y to torch.float
    y = y.to(device)

    # Forward pass
    y_pred = model(x)

    # Compute the loss
    loss = loss_fn(y_pred, y)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


c:\Users\rifat\miniconda3\envs\tf\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: Could not infer dtype of NoneType

In [19]:
dataloader.dataset

,cpu_uses,memory_uses,Wi_fi_uses,disc_uses,Type
0,0.000000,0.000000e+00,0.0,0.0,1.0
1,0.000000,3.176540e-03,0.0,0.0,0.0
2,0.000000,9.979335e-01,0.0,0.0,1.0
3,0.000000,5.353317e-04,0.0,0.0,0.0
4,0.000000,6.089447e-02,0.0,0.0,0.0
5,0.000000,7.464073e-04,0.0,0.0,0.0
6,0.000000,8.878832e-05,0.0,0.0,0.0
7,0.000000,2.407529e-02,0.0,0.0,0.0
8,0.000000,0.000000e+00,0.0,0.0,0.0
9,0.000000,2.718939e-02,0.0,0.0,0.0


In [35]:
y_pred = model(x)

In [37]:
y_pred

tensor([[ 0.0470],
        [-0.0080],
        [ 0.0585],
        [-0.0079],
        [-0.0081]], grad_fn=<TanhBackward0>)

In [40]:
class CustomDataset(Dataset):
    def __init__(self, df, num_columns):
        self.df = df
        self.num_columns = num_columns

    def _custom_logic(self, subset):

        target_value = subset.sum().values.sum()
        return target_value

    def __len__(self):
        return len(self.df) - self.num_columns + 1

    def __getitem__(self, idx):
        start_row = idx * self.num_columns
        end_row = (idx + 1) * self.num_columns
        subset = self.df.iloc[start_row:end_row]
        y = self._custom_logic(subset)
        x = subset.values
        return torch.tensor(x), torch.tensor(y)


In [ ]:
NUM_ROWS=10
NUM_COLS=5

# Select the desired subset of data chronologically
selected_data = df.iloc[:NUM_ROWS, :NUM_COLS]
# Convert the selected data to a NumPy array
data_array = selected_data.values